In [1]:
from matplotlib import pyplot as plt
import os
from skimage import io, transform, util, img_as_float
from skimage.filters import gaussian
from skimage.color import separate_stains, hax_from_rgb, rgb2gray
from skimage.exposure import rescale_intensity
from skimage.util import img_as_ubyte, crop
from skimage.draw import rectangle
from skimage.morphology import remove_small_objects, remove_small_holes
from skimage.registration import phase_cross_correlation
from skimage.measure import regionprops_table
from skimage.segmentation import expand_labels, find_boundaries

import math
import numpy as np
import tifffile
import cv2
import glob
import numpy as np
import napari
import time
import re

from joblib import Parallel, delayed
from itertools import groupby
from tqdm import tqdm
import pandas as pd
import multiprocessing
available_cores = multiprocessing.cpu_count()

from stardist.models import StarDist2D
from stardist import random_label_cmap
from csbdeep.utils import normalize
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 10, 10
plt.rcParams['image.cmap'] = 'gray'

%load_ext autotime

time: 201 µs (started: 2022-01-06 13:20:56 -06:00)


In [2]:
# Set dev_mode to True to print step by step outputs to confirm pipeline is working as expected and find and correct issues
dev_mode = True
# Set batch_mode to False to run workflow on a single slide - ideal for setting up workflow for the first time. Set to True to run multiple slides at the same time.
batch_mode = False

time: 657 µs (started: 2022-01-06 13:21:06 -06:00)


In [3]:
#set pipeline output directory
output_dir = '/home/workstation/Python_Output'
#set image input directory
raw_img_dir = '/media/workstation/Backup Drive/Ihrie_MxIHC_2020/SCN'  # for each round of imaging, images should be saved by the slide name and placed in a folder named for the marker that has been stained in the contained images
# input order of MxIHC staining
stain_order = ['PD-1', 'PD-L1', 'CD68', 'CD3', 'FoxP3', 'Iba-1', 'CD8', 'CD4']   # order in which antibodies were stained
# input patient IDs
slide_id = ['N14-248-1B', 'N14-692-2A', 'N14-945-2A', 'N15-501-1B', 'N16-244-2A', 'N16-478-2E']   
# Stardist Model Directory
stardist_dir = '/home/workstation/Dropbox (VU Basic Sciences)/Python Scripts/From ubuntu'


time: 889 µs (started: 2022-01-06 13:21:07 -06:00)


In [4]:
# Setting variables

#BaseAligned image input
input_folder = output_dir + '/BaseAligned'
#Tiles output
output_folder = output_dir + '/Tiled'
# registered stack output
reg_folder = output_dir + '/Registered'
# cropped stack output
crops_folder = output_dir + '/CroppedStacks'
# reassembled registered slides output
stitch_folder = output_dir + '/StitchedSlides'
# nuclear labels output if using built in Stardist, input if importing segmentation
labels_folder = output_dir + '/Labels'
# plot outputs folder
plots_folder = output_dir + '/Plots'

tile_x, tile_y = 2048, 2048              # Can Edit Tile Size Here
overlap_x, overlap_y = 512, 512          # Can Edit Overlap Here

background_int = 250                     # Can Edit Mean Pixel Intensity for Tissue Detection Here (Sets mean pixel intensity value at which tiles are either discarded are saved during image tiling - saved blocks are noted with blue text in the tilemaps while discarded blocks are noted with red text)
                                         # If tiles containing substantial tissue are discarded, adjust value higher, if tiles with no tissue are saved, adjust value lower

_ransacReprojThreshold = 3               # Can Edit Maximum allowed reprojection error to treat a point pair as an inlier (used in the RANSAC method only) for Keypoint registration

_num_good = 10                           # Can Edit Minimum number of good matches needed for keypoint registration to be considered successful
_averageDistance = 0.5                   # Can Edit Maximum average distance of keypoint matches for registration to be considered successful


stardist_segmentation = True            # Set to True if using stardist segmentation model, False if labels files will be generated and uploaded separately 

exp_lab_dist = 10                        # Can Edit pixel distance to expand nuclear labels to capture cytoplasmic signal

time: 1.34 ms (started: 2022-01-06 13:21:09 -06:00)


# Define Functions #


In [5]:
def closest(lst, K):    
    return lst.index(lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))])


def goalRes(file):
    l_res = []
    for index, page in enumerate(tifffile.TiffFile(file).pages):
        try:
            l_res.append([index, page.tags['XResolution'].value[1]])
        except:
            l_res.append([index, 0])    # catches pages where XResolution is not a valid tag
    return closest([x[1] for x in l_res], 237000)   # this value is currently set specifically to return the 20X resolution image from specific SCN file type we get from DHSR
    


time: 1.8 ms (started: 2022-01-06 13:21:24 -06:00)


In [6]:
def deconvReg(image):
    if type(image) == str:
        img = io.imread(image)
        hax  = separate_stains(img, hax_from_rgb)
        hema = hax[:,:,0]
        h = rescale_intensity(hema, in_range = (0, 0.1), out_range = 'uint8')  # in_range set to 0.1 here to get a stronger hema signal for better registration
    elif type(image) == np.ndarray:
        hax = separate_stains(image, hax_from_rgb)
        hema = hax[:,:,0]
        h = rescale_intensity(hema, in_range = (0, 0.1), out_range = 'uint8')
    return h

time: 1.08 ms (started: 2022-01-06 13:21:25 -06:00)


In [7]:
def base_align(base):
    l_sizes =  []
    test_img = io.imread(base[0])
    for n in range(8):
        l_sizes.append(int(test_img.shape[0]/2**n) * int(test_img.shape[1]/2**n))
    scale_down = 2**closest(l_sizes, 3000000)
    del test_img
    
    l_sim = []
    if dev_mode == True:
        f = open('{}/{}_distance.txt'.format(input_folder, os.path.splitext(os.path.basename(base[0]))[0]), 'w+')  
    os.chdir(output_dir)
    original_base_0 = io.imread(base[0])
    tifffile.imwrite(output_dir + '/BaseAligned/Aligned_{}'.format(os.path.basename(base[0])), original_base_0)    # saves fixed image (untransformed)
    base_0_gray = img_as_ubyte(rgb2gray(cv2.resize(original_base_0, (int(original_base_0.shape[1]/scale_down), int(original_base_0.shape[0]/scale_down)))))


    for index, x in enumerate(base[1:]):
        original_moving = io.imread(x)
        moving_gray = img_as_ubyte(rgb2gray(cv2.resize(original_moving, (int(original_moving.shape[1]/scale_down), int(original_moving.shape[0]/scale_down)))))

        
        fd = cv2.KAZE_create(extended=True)
        try:
            moving_pts, target_pts, averageDistance, num_good = match_keypoints(moving_gray, base_0_gray, feature_detector=fd)
            
            transformer = transform.EuclideanTransform()
            try:
                transformer.estimate(target_pts * scale_down, moving_pts * scale_down)
                output_shape_rc = original_base_0.shape[:2]
                warped_img = transform.warp(original_moving, transformer, output_shape=output_shape_rc)
            except:
                if dev_mode == True:
                    f.write('registration failure on ' + str(index+1) + ' apply_transform')
                    f.close()
                break
            warped_img = img_as_ubyte(warped_img)
            tifffile.imwrite(output_dir + '/BaseAligned/Aligned_{}'.format(os.path.basename(x)), warped_img)  # save transformed image    
            warped_gray = img_as_ubyte(rgb2gray(cv2.resize(warped_img, (int(warped_img.shape[1]/scale_down), int(warped_img.shape[0]/scale_down)))))

            gaus_base = gaussian(base_0_gray, 2)
            gaus_warped = gaussian(warped_gray, 2)
            gaus_moving = gaussian(moving_gray, 2)
            X = metrics.structural_similarity(gaus_base, gaus_warped)
            Y = metrics.structural_similarity(gaus_base, gaus_moving)
            X1 = normalized_mutual_information(gaus_base, gaus_warped, bins = 100)
            Y1 = normalized_mutual_information(gaus_base, gaus_moving, bins = 100)

            l_sim.append([os.path.basename(x), X, Y, X1, Y1])

            if dev_mode == True:
                f.write('{}_Average Distance {} = {}________Number of Good Matches = {}\n'.format(str(index+1),str(averageDistance),os.path.basename(x) ,str(num_good)))
        except:
            if dev_mode == True:
                f.write('registration failure on ' + str(index+1) + ' match_keypoints.\n')
    df = pd.DataFrame(l_sim, columns = ['moving_file', 'warped_sim', 'moving_sim', 'warped_nmi', 'moving_nmi'])
    df.to_csv(output_dir + '/BaseAligned/Similarity_{}.csv'.format(os.path.splitext(os.path.basename(x))[0]), index = False)
    if dev_mode == True:
        f.close()    

time: 2.22 ms (started: 2022-01-06 13:21:25 -06:00)


In [8]:
def tileSave(file):
    tissue_detect = []
    image = io.imread(file)
    tile_rows_overlap = list(range(int(image.shape[0]/tile_x)))
    tile_cols_overlap = list(range(int(image.shape[1]/tile_y)))
    combined = [(f,s) for f in tile_rows_overlap for s in tile_cols_overlap]    
    filename = os.path.basename(os.path.splitext(file)[0])   
    _, _, _, pat, stain = filename.split('_')

    os.makedirs('{}/{}'.format(output_folder, filename), exist_ok = True)
    os.chdir('{}/{}'.format(output_folder,filename))


    log = open('savelog.txt', 'w+')
    
    # Create low res slide image with tile rows and columns labeled
    
    imgScale = 0.25   # scale factor for low res slide image
    newX, newY = int(image.shape[1]*imgScale), int(image.shape[0]*imgScale)
    lowres = cv2.resize(image, (newX, newY))
       
     # Save cropped tiles

    for i, j in combined: 
        row_top_offset = max(0, i * tile_x - overlap_x)
        row_bot_offset = max(0, image.shape[0] - (tile_x * (i+1) + overlap_x))
        col_l_offset = max(0, j * tile_y - overlap_y)
        col_r_offset = max(0, image.shape[1] - (tile_y * (j+1) + overlap_y))  
        croppedimg = crop(image,((row_top_offset,row_bot_offset),(col_l_offset, col_r_offset),(0,0)), copy = False)   
        #tissue detection block
        croppedimg[np.where((croppedimg==[0,0,0]).all(axis=2))] = [255,255,255]   # Turns black pixels white
        tissue_detect.append([tile_rows_overlap[i], tile_cols_overlap[j], np.mean(croppedimg)])
        TEXT = '({}, {})'.format(i, j)
        TEXT_SCALE = 3                                                # font sizes may need to be adjusted if input image size change
        TEXT_THICKNESS = 3
        TEXT_FACE = 3
        text_size, _ = cv2.getTextSize(TEXT, TEXT_FACE, TEXT_SCALE, TEXT_THICKNESS)
        LOC = (int(j*tile_x*imgScale + (tile_x*imgScale)/2), int(i*tile_y*imgScale + (tile_y*imgScale)/2))
        TEXT_ORG = (int(LOC[0]-text_size[0]/2), int(LOC[1] + text_size[1]/2))        
        if np.mean(croppedimg) > background_int:    # if mean pixel intensity is less than background_int (default 250) (less white than 250, 250, 250) then saves tile.  otherwise tile is considered 'background' and discarded.  this value should be tweaked based on typical image background 
            alpha = 230
            out_thick = 5
            color = (alpha, alpha, alpha)
            TL = (int(i*tile_x*imgScale), int(j*tile_x*imgScale))
            TR = (int(i*tile_x*imgScale), int((j+1)*tile_x*imgScale))
            BR = (int((i+1)*tile_x*imgScale), int((j+1)*tile_x*imgScale))
            BL = (int((i+1)*tile_x*imgScale), int(j*tile_x*imgScale))

            rr, cc = rectangle(start = TL, end =(TR[0] + out_thick, TR[1]), shape = lowres.shape)
            lowres[rr,cc] = color
            rr, cc = rectangle(start = TR, end = (BR[0], BR[1]-out_thick), shape = lowres.shape)
            lowres[rr,cc] = color
            rr, cc = rectangle(start = BL, end = (BR[0] - out_thick, BR[1]), shape = lowres.shape)
            lowres[rr,cc] = color
            rr,cc = rectangle(start = TL, end = (BL[0], BL[1] + out_thick), shape = lowres.shape)
            lowres[rr,cc] = color
            cv2.putText(lowres, text = TEXT, org = TEXT_ORG, fontFace = TEXT_FACE, fontScale = TEXT_SCALE, color = color, thickness = TEXT_THICKNESS)     
    for i, j in combined:
        row_top_offset = max(0, i * tile_x - overlap_x)
        row_bot_offset = max(0, image.shape[0] - (tile_x * (i+1) + overlap_x))
        col_l_offset = max(0, j * tile_y - overlap_y)
        col_r_offset = max(0, image.shape[1] - (tile_y * (j+1) + overlap_y))  
        croppedimg = crop(image,((row_top_offset,row_bot_offset),(col_l_offset, col_r_offset),(0,0)), copy = False)   
        #tissue detection block
        croppedimg[np.where((croppedimg==[0,0,0]).all(axis=2))] = [255,255,255]   # Turns black pixels white
        tissue_detect.append([tile_rows_overlap[i], tile_cols_overlap[j], np.mean(croppedimg)])
        TEXT = '({}, {})'.format(i, j)
        TEXT_SCALE = 3                                                # font sizes may need to be adjusted if input image size change
        TEXT_THICKNESS = 3
        TEXT_FACE = 3
        text_size, _ = cv2.getTextSize(TEXT, TEXT_FACE, TEXT_SCALE, TEXT_THICKNESS)
        LOC = (int(j*tile_x*imgScale + (tile_x*imgScale)/2), int(i*tile_y*imgScale + (tile_y*imgScale)/2))
        TEXT_ORG = (int(LOC[0]-text_size[0]/2), int(LOC[1] + text_size[1]/2))        

        if np.mean(croppedimg) < background_int:    # if mean pixel intensity is less than background_int (default 250) (less white than 250, 250, 250) then saves tile.  otherwise tile is considered 'background' and discarded.  this value should be tweaked based on typical image background 
            io.imsave('{}_{}_r{}_c{}.tiff'.format(pat, stain, tile_rows_overlap[i], tile_cols_overlap[j]), croppedimg)
            cv2.putText(lowres, text = TEXT, org = TEXT_ORG, fontFace = TEXT_FACE, fontScale = TEXT_SCALE, color = (0,0,0), thickness = TEXT_THICKNESS)  
            alpha = 0
            out_thick = 5
            color = (alpha, alpha, alpha)
            TL = (int(i*tile_x*imgScale), int(j*tile_x*imgScale))
            TR = (int(i*tile_x*imgScale), int((j+1)*tile_x*imgScale))
            BR = (int((i+1)*tile_x*imgScale), int((j+1)*tile_x*imgScale))
            BL = (int((i+1)*tile_x*imgScale), int(j*tile_x*imgScale))

            rr, cc = rectangle(start = (TL[0]-out_thick, TL[1]-out_thick), end =(TR[0] + out_thick, TR[1]-out_thick), shape = lowres.shape)
            lowres[rr,cc] = color
            rr, cc = rectangle(start = (TR[0]+out_thick, TR[1]-out_thick), end = (BR[0]+out_thick, BR[1]+out_thick), shape = lowres.shape)
            lowres[rr,cc] = color
            rr, cc = rectangle(start = (BL[0]-out_thick, BL[1]+out_thick), end = (BR[0] + out_thick, BR[1]+out_thick), shape = lowres.shape)
            lowres[rr,cc] = color
            rr,cc = rectangle(start = (TL[0]-out_thick, TL[1]-out_thick), end = (BL[0]-out_thick, BL[1] + out_thick), shape = lowres.shape)
            lowres[rr,cc] = color  
    io.imsave('{}/{}_{}_tilemap.tiff'.format(output_folder, pat, stain), lowres)
    df_tissue_detect = pd.DataFrame(tissue_detect, columns = ['row', 'col', 'mean_int'])
    df_tissue_detect.to_csv('{}/{}_{}_tissue_detect.csv'.format(output_folder, pat, stain), index = False)
    log.close()

time: 3.1 ms (started: 2022-01-06 13:21:26 -06:00)


In [9]:
def match_keypoints(moving, target, feature_detector):

    kp1, desc1 = feature_detector.detectAndCompute(moving, None)
    kp2, desc2 = feature_detector.detectAndCompute(target, None)

    matcher = cv2.BFMatcher(normType=cv2.NORM_L2, crossCheck=True)
    matches = matcher.match(desc1, desc2)
    matches_sorted = sorted(matches, key = lambda x: x.distance)
    
    totalDistance = 0
    for g in matches_sorted:
        totalDistance += g.distance
        
    if len(matches_sorted) == 0:
        averageDistance = 0
    else:
        averageDistance = totalDistance/len(matches_sorted)


    src_match_idx = [m.queryIdx for m in matches_sorted[:200]]   # list only first 200 matches can edit to select more or less
    dst_match_idx = [m.trainIdx for m in matches_sorted[:200]]

    src_points = np.float32([kp1[i].pt for i in src_match_idx])
    dst_points = np.float32([kp2[i].pt for i in dst_match_idx])

    H, mask = cv2.findHomography(src_points, dst_points, cv2.RANSAC, ransacReprojThreshold=_ransacReprojThreshold)   # original value = 7 

    good = [matches_sorted[i] for i in np.arange(0, len(mask)) if mask[i] == [1]]
    
    num_good = len(good) # count how many good matches were found

    filtered_src_match_idx = [m.queryIdx for m in good]
    filtered_dst_match_idx = [m.trainIdx for m in good]

    filtered_src_points = np.float32([kp1[i].pt for i in filtered_src_match_idx])
    filtered_dst_points = np.float32([kp2[i].pt for i in filtered_dst_match_idx])

    return filtered_src_points, filtered_dst_points, averageDistance, num_good

def apply_transform(moving, target, moving_pts, target_pts, transformer, output_shape_rc=None):

    if output_shape_rc is None:
        output_shape_rc = target.shape[:2]

    if str(transformer.__class__) == "<class 'skimage.transform.EuclideanTransform'>":
        transformer.estimate(target_pts, moving_pts)
        warped_img = transform.warp(moving, transformer, output_shape=output_shape_rc)

        ### Restimate to warp points
        transformer.estimate(moving_pts, target_pts)
        warped_pts = transformer(moving_pts)
    else:
        transformer.estimate(moving_pts, target_pts)
        warped_img = transform.warp(moving, transformer.inverse, output_shape=output_shape_rc)
        warped_pts = transformer(moving_pts)

    return warped_img, warped_pts

def keypoint_distance(moving_pts, target_pts, img_h, img_w):
    dst = np.sqrt(np.sum((moving_pts - target_pts)**2, axis=1)) / np.sqrt(img_h**2 + img_w**2)
    return np.mean(dst)

time: 2.12 ms (started: 2022-01-06 13:21:26 -06:00)


In [10]:
def regAll(path):
    os.makedirs('{}/{}'.format(
        reg_folder,
        os.path.basename(path)), exist_ok = True)
    os.chdir('{}/{}'.format(
        reg_folder,
        os.path.basename(path)))
    if dev_mode == True:
        f = open('{}/{}/distance.txt'.format(
                reg_folder,
                os.path.basename(path)), 'w+')   
    
    d_reg = {}
    for i, k in enumerate(all_paths[path]):      
        # block for registering image to stain_order0

        original_target = io.imread(all_paths[path][0])  # rgb input
        original_moving = io.imread(k)  # rgb

        target_file = deconvReg(all_paths[path][0])   # Hema only
        moving_file = deconvReg(k)   # Hema only

        target = img_as_ubyte(gaussian(target_file, 3))    #blur on hema
        moving = img_as_ubyte(gaussian(moving_file, 3))    #blur on hema
                                                           #gaussian alpha can be tuned here
        _, stainord, _, slide, stain = os.path.basename(os.path.dirname(k)).split('_')
        _, _, row, col = os.path.basename(os.path.splitext(k)[0]).split('_')

        fd = cv2.KAZE_create(extended=True)
        try:
            moving_pts, target_pts, averageDistance, num_good = match_keypoints(moving, target, feature_detector=fd)
        except:
            if dev_mode == True:
                f.write('registration failure on' + str(i) + 'match_keypoints')
                f.close()
            break
        if dev_mode == True:
            f.write('{}_Average Distance {} = {}________Number of Good Matches = {}\n'.format(str(i),str(averageDistance), all_paths[path][i],str(num_good)))

        transformer = transform.EuclideanTransform()
        try:
            warped_img, warped_pts = apply_transform(original_moving, original_target, moving_pts, target_pts, transformer=transformer)
        except:
            if dev_mode == True:
                f.write('registration failure on' + str(i) + 'apply_transform')
                f.close()
            break

        warped_img = img_as_ubyte(warped_img)
        d_reg[int(stainord),'i0'] = warped_img 
        if dev_mode == True:
            io.imsave(str(i) + '_i0_' + (os.path.basename(os.path.splitext(all_paths[path][i])[0])+ '_reg.tiff'), warped_img)

        
        
        # if a suboptimal registration is detected via low number of good keypoint matches or large average distance between matches
        # instead of registering to the index0 image, attemp to register to the index -1 image (previous image in registration stack)
        if num_good < _num_good or averageDistance > _averageDistance:  
            
            if (int(stainord)-1, 'i-1') in d_reg.keys():
                original_target = d_reg[int(stainord)-1, 'i-1']
            else:
                original_target = d_reg[int(stainord)-1, 'i0']
                
            if (int(stainord)-1, 'i-1') in d_reg.keys():
                target_file = deconvReg(d_reg[int(stainord)-1, 'i-1'])
            else:
                target_file = deconvReg(d_reg[int(stainord)-1, 'i0'])


            target = img_as_ubyte(gaussian(target_file, 6))    #alpha could be tuned for best performace 
            moving = img_as_ubyte(gaussian(moving_file, 6))


            fd = cv2.KAZE_create(extended=True)
            try:
                moving_pts, target_pts, averageDistance, num_good = match_keypoints(moving, target, feature_detector=fd)
            except:
                if dev_mode == True:
                    f.write('fail on' + str(i) + '-1 match_keypoints')
                    f.close()
                break
            if dev_mode == True:
                f.write('{}_i-1_Average Distance {} = {}________Number of Good Matches = {}\n'.format(str(i),str(averageDistance), all_paths[path][i],str(num_good)))

            transformer = transform.EuclideanTransform()
            try:
                warped_img2, warped_pts2 = apply_transform(original_moving, original_target, moving_pts, target_pts, transformer=transformer)
            except:
                if dev_mode == True:
                    f.write('fail on' + str(i) + '-1 apply_transform')
                    f.close()
                break
            warped_img2 = img_as_ubyte(warped_img2)
            if num_good<10 or averageDistance > 0.5:
                break
            else:
                d_reg[int(stainord), 'i-1'] = warped_img2
                if dev_mode == True:
                    io.imsave(str(i)+'_i-1_' + (os.path.basename(os.path.splitext(all_paths[path][i])[0]+ '_reg.tiff')), warped_img2)

                continue
            break
    if dev_mode == True:
        f.close()


    
    #combine mask on d_reg
    l_reg_sort = [list(g) for k, g in groupby(list(d_reg.keys()), key = lambda x: x[0])]  # sort files by first character (original and i-1 image will both have same number preceding) # need to update to allow for numbers > 10
    l_reg_filter = []
    for h in l_reg_sort:         # if i-1 image exists, take it if not take the regular registered image
        if len(h) == 1:
            l_reg_filter.append(d_reg[h[0]])
        if len(h) == 2:
            l_reg_filter.append(d_reg[h[1]])

    combined_mask = np.zeros((l_reg_filter[0].shape[0],l_reg_filter[0].shape[1]), dtype=bool)    # create empty mask file with same shape as images
    for k in l_reg_filter:
        if len(l_reg_filter) == len(stain_order):     # only take tiles where we have an image for every stain in stain_order
            k[np.all(k == (0,0,0), axis = -1)] = (255, 255,255)  # set black registration gaps to white
            gaus = gaussian(k, 6, multichannel = True)  # apply gaussian blur to image, returns float

            thresh = 0.97    # Try range of values to determine ideal threshold
            mask_gaus = (gaus[:,:,1] ==0) | ((gaus[:,:,0] > thresh) | (gaus[:,:,1] > thresh) | (gaus[:,:,2] > thresh)) # masks any image area where pixel value is either 0 or above threshold on any of the 3 channels
            combined_mask = combined_mask + mask_gaus
        else:
            break
    else:     
        clean_mask = remove_small_objects(combined_mask, min_size=2000)   # can edit size of small objects / small holes here
        clean_mask = remove_small_holes(clean_mask, area_threshold=2000)
        if dev_mode == True:
            io.imsave('{}_{}_{}_CombinedMask.tiff'.format(slide, row, col), img_as_ubyte(clean_mask))   #save combined mask file
        arrays = []
        for y in l_reg_filter:
            y[np.all(y == (0,0,0), axis = -1)] = (250, 250,250)
            y[clean_mask] = 250
            arrays.append(y)
        stack = np.stack(arrays, axis = 0)
        io.imsave('{}_{}_{}_stack.tiff'.format(slide, row, col), stack)
        if dev_mode == True:
            demo = l_reg_filter[0]   # apply mask to index 0 stain as example       
            gaps = demo[:,:,1] == 0
            demo[gaps] = 250
            demo[clean_mask] = 250
            io.imsave('{}_{}_{}_demo.tiff'.format(slide, row, col), demo)   # save an example of the mask on image

time: 2.72 ms (started: 2022-01-06 13:21:27 -06:00)


# Base Alignment of Full Slide Images #

In [11]:
# create subfolders in output folder
os.chdir(output_dir)
os.makedirs(output_dir + '/BaseImages', exist_ok = True)
os.makedirs(input_folder, exist_ok = True)
os.makedirs(stitch_folder, exist_ok = True)
os.makedirs(crops_folder, exist_ok = True)
os.makedirs(labels_folder, exist_ok = True)
os.makedirs(plots_folder, exist_ok = True)

time: 2.31 ms (started: 2022-01-06 13:21:28 -06:00)


In [12]:
# create list of lists of paths to raw images for each slide stains in imaged order
all_slides = []
for y in slide_id:
    slide_names = []
    for root, dirs, files in os.walk(raw_img_dir):
        for i in files:
            if os.path.splitext(i)[1] in ['.scn', '.svs']:
                if os.path.splitext(i)[0] == y:
                    stain = os.path.basename(os.path.dirname(os.path.join(root, i)))
                    stain_ord = stain_order.index(stain)
                    slide_names.append(tuple([stain_ord, os.path.join(root, i)]))

    slide_names = sorted(slide_names)
    slide_names = [x[1] for x in slide_names]
    all_slides.append(slide_names)

#checks that for each sample, raw image file exists for every file and are in the correct order
if dev_mode == True:
    for index, l in enumerate(all_slides):
        for indexx, ll in enumerate(l):
            print('Expected ' + stain_order[indexx] +' '+ slide_id[index] + ' file = ' +  ll)
            assert os.path.basename(os.path.dirname(ll)) == stain_order[indexx] 


Expected PD-1 N14-248-1B file = /media/workstation/Backup Drive/Ihrie_MxIHC_2020/SCN/PD-1/N14-248-1B.scn
Expected PD-L1 N14-248-1B file = /media/workstation/Backup Drive/Ihrie_MxIHC_2020/SCN/PD-L1/N14-248-1B.scn
Expected CD68 N14-248-1B file = /media/workstation/Backup Drive/Ihrie_MxIHC_2020/SCN/CD68/N14-248-1B.scn
Expected CD3 N14-248-1B file = /media/workstation/Backup Drive/Ihrie_MxIHC_2020/SCN/CD3/N14-248-1B.scn
Expected FoxP3 N14-248-1B file = /media/workstation/Backup Drive/Ihrie_MxIHC_2020/SCN/FoxP3/N14-248-1B.scn
Expected Iba-1 N14-248-1B file = /media/workstation/Backup Drive/Ihrie_MxIHC_2020/SCN/Iba-1/N14-248-1B.scn
Expected CD8 N14-248-1B file = /media/workstation/Backup Drive/Ihrie_MxIHC_2020/SCN/CD8/N14-248-1B.scn
Expected CD4 N14-248-1B file = /media/workstation/Backup Drive/Ihrie_MxIHC_2020/SCN/CD4/N14-248-1B.scn
Expected PD-1 N14-692-2A file = /media/workstation/Backup Drive/Ihrie_MxIHC_2020/SCN/PD-1/N14-692-2A.scn
Expected PD-L1 N14-692-2A file = /media/workstation/Bac

In [43]:
# Choose slide to run pipeline on
if batch_mode == False:
    for index, slide in enumerate(slide_id):
        print('slide index = '+str(index) + ' slide_id = ' +slide)

    test_slide_index = 3   # if batch_mode = False, set index of single slide to run workflow on here

slide index = 0 slide_id = N14-248-1B
slide index = 1 slide_id = N14-692-2A
slide index = 2 slide_id = N14-945-2A
slide index = 3 slide_id = N15-501-1B
slide index = 4 slide_id = N16-244-2A
slide index = 5 slide_id = N16-478-2E
time: 810 µs (started: 2022-01-06 16:39:10 -06:00)


In [44]:
# Open and resave raw images cropped to the nearest multiple of tile_x and tile_y pixels in x, y dimensions
if batch_mode == False:
    for y in [all_slides[test_slide_index]]:
        l_shapes = []
        for index, slide in enumerate(y):
            img = tifffile.imread(slide, key = goalRes(slide))    # watch out for if within an image set, output shapes change
            crop_row = (img.shape[0]%tile_x)
            crop_col = (img.shape[1]%tile_y)
            out_image = crop(img, ((int(math.floor(crop_row/2)), int(math.ceil(crop_row/2))), (int(math.floor(crop_col/2)), int(math.ceil(crop_col/2))), (0,0)), copy = False)
            if dev_mode == True:
                print(slide , out_image.shape)
            l_shapes.append(out_image.shape)
            tifffile.imwrite(output_dir + f'/BaseImages/{index}_Base_{os.path.basename(os.path.splitext(slide)[0])}_{os.path.basename(os.path.dirname(slide))}.tiff', out_image, photometric='minisblack') 
        assert len(set(l_shapes))<=1 # ensures that all images are the same dimensions
else: 
    for y in all_slides:
        l_shapes = []
        for index, slide in enumerate(y):
            img = tifffile.imread(slide, key = goalRes(slide))    # watch out for if within an image set, output shapes change
            crop_row = (img.shape[0]%tile_x)
            crop_col = (img.shape[1]%tile_y)
            out_image = crop(img, ((int(math.floor(crop_row/2)), int(math.ceil(crop_row/2))), (int(math.floor(crop_col/2)), int(math.ceil(crop_col/2))), (0,0)), copy = False)
            if dev_mode == True:
                print(slide , out_image.shape)
            l_shapes.append(out_image.shape)
            tifffile.imwrite(output_dir + f'/BaseImages/{index}_Base_{os.path.basename(os.path.splitext(slide)[0])}_{os.path.basename(os.path.dirname(slide))}.tiff', out_image, photometric='minisblack') 
        assert len(set(l_shapes))<=1 # ensures that all images are the same dimensions

/media/workstation/Backup Drive/Ihrie_MxIHC_2020/SCN/PD-1/N15-501-1B.scn (40960, 28672, 3)
/media/workstation/Backup Drive/Ihrie_MxIHC_2020/SCN/PD-L1/N15-501-1B.scn (40960, 28672, 3)
/media/workstation/Backup Drive/Ihrie_MxIHC_2020/SCN/CD68/N15-501-1B.scn (40960, 28672, 3)
/media/workstation/Backup Drive/Ihrie_MxIHC_2020/SCN/CD3/N15-501-1B.scn (40960, 28672, 3)
/media/workstation/Backup Drive/Ihrie_MxIHC_2020/SCN/FoxP3/N15-501-1B.scn (40960, 28672, 3)
/media/workstation/Backup Drive/Ihrie_MxIHC_2020/SCN/Iba-1/N15-501-1B.scn (40960, 28672, 3)
/media/workstation/Backup Drive/Ihrie_MxIHC_2020/SCN/CD8/N15-501-1B.scn (40960, 28672, 3)
/media/workstation/Backup Drive/Ihrie_MxIHC_2020/SCN/CD4/N15-501-1B.scn (40960, 28672, 3)
time: 39.2 s (started: 2022-01-06 16:39:13 -06:00)


In [45]:
l_base = []
for y in slide_id:
    slide_names = []
    for files in os.listdir(output_dir + '/BaseImages'):
        _, _, pat, stain = os.path.splitext(files)[0].split('_')
        if pat == y:
            stain_ord = stain_order.index(stain)
            slide_names.append(tuple([stain_ord, os.path.join(output_dir+ '/BaseImages/' ,files)]))
    slide_names = sorted(slide_names)
    slide_names = [x[1] for x in slide_names]
    l_base.append(slide_names)

time: 1.72 ms (started: 2022-01-06 16:39:52 -06:00)


In [46]:
if batch_mode == False:
    base_align(l_base[test_slide_index])
else:
    Parallel(n_jobs = 1)(delayed(base_align)(file) for file in tqdm(l_base))   # decrease n_jobs if running out of memory on this step

time: 18min 4s (started: 2022-01-06 16:39:52 -06:00)


In [47]:
# collects all base_aligned files 
l_files = sorted(glob.glob(input_folder + '/*.tiff'))

time: 968 µs (started: 2022-01-06 16:57:57 -06:00)


In [33]:
# dev mode:open base aligned images in napari 
if dev_mode == True and batch_mode == False:
    with napari.gui_qt():
        viewer = napari.view_image(io.imread(l_files[0]), name = os.path.basename(l_files[0]))
        for file in l_files[1:]:
            viewer.add_image(io.imread(file), name = os.path.basename(file))

/home/workstation/tensor_env/lib/python3.8/site-packages/napari/_vispy/vispy_image_layer.py:183: UserWarning: data shape (40960, 32768, 3) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
/home/workstation/tensor_env/lib/python3.8/site-packages/napari/_vispy/vispy_image_layer.py:183: UserWarning: data shape (30720, 16384, 3) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
/home/workstation/tensor_env/lib/python3.8/site-packages/napari/_vispy/vispy_image_layer.py:183: UserWarning: data shape (40960, 32768, 3) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
/home/workstation/tensor_env/lib/python3.8/site-packages/napari/_vispy/vispy_image_layer.py:183: UserWarning: data shape (30720, 16384, 3) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least on

time: 8min 3s (started: 2022-01-06 14:24:00 -06:00)


# Tile Image and Register #

In [34]:
l_files

['/home/workstation/Python_Output/BaseAligned/Aligned_0_Base_N14-945-2A_PD-1.tiff',
 '/home/workstation/Python_Output/BaseAligned/Aligned_0_Base_N16-244-2A_PD-1.tiff',
 '/home/workstation/Python_Output/BaseAligned/Aligned_1_Base_N14-945-2A_PD-L1.tiff',
 '/home/workstation/Python_Output/BaseAligned/Aligned_1_Base_N16-244-2A_PD-L1.tiff',
 '/home/workstation/Python_Output/BaseAligned/Aligned_2_Base_N14-945-2A_CD68.tiff',
 '/home/workstation/Python_Output/BaseAligned/Aligned_2_Base_N16-244-2A_CD68.tiff',
 '/home/workstation/Python_Output/BaseAligned/Aligned_3_Base_N14-945-2A_CD3.tiff',
 '/home/workstation/Python_Output/BaseAligned/Aligned_3_Base_N16-244-2A_CD3.tiff',
 '/home/workstation/Python_Output/BaseAligned/Aligned_4_Base_N14-945-2A_FoxP3.tiff',
 '/home/workstation/Python_Output/BaseAligned/Aligned_4_Base_N16-244-2A_FoxP3.tiff',
 '/home/workstation/Python_Output/BaseAligned/Aligned_5_Base_N14-945-2A_Iba-1.tiff',
 '/home/workstation/Python_Output/BaseAligned/Aligned_5_Base_N16-244-2A_I

time: 2.62 ms (started: 2022-01-06 14:32:03 -06:00)


In [48]:
# Crop and save all tiles from all base aligned images
Parallel(n_jobs = int(available_cores * 0.9))(delayed(tileSave)(file)for file in tqdm(l_files))

100%|██████████| 24/24 [00:00<00:00, 10740.86it/s]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

time: 4min 7s (started: 2022-01-06 16:57:57 -06:00)


In [36]:
# list tilemaps and open in napari
if dev_mode == True and batch_mode == False:
    l_tilemaps = sorted(glob.glob(output_folder + '/*_tilemap.tiff'))  
    with napari.gui_qt():
        viewer = napari.view_image(io.imread(l_tilemaps[0]), name = os.path.basename(l_tilemaps[0]))
        for file in l_tilemaps[1:]:
            viewer.add_image(io.imread(file), name = os.path.basename(file))

time: 3min 3s (started: 2022-01-06 14:35:13 -06:00)


In [49]:
# list all tiles for a slide
l_allfiles = []
for root, dirs, files in os.walk(output_folder):
    for file in files:
        if file.endswith('_reg.tiff') | file.endswith('_tilemap.tiff'):
            continue
        if os.path.basename(root) == 'mask':
            continue
        if file.endswith('.tiff'):
            l_allfiles.append(os.path.join(root,file))

time: 21.2 ms (started: 2022-01-06 17:02:04 -06:00)


In [50]:
#Extract metadata for each cropped image from file names
metadata = []
for path in l_allfiles:
    file = os.path.basename(os.path.splitext(path)[0])
    slide,stain,row,col = file.split('_')
    coord = row+col
    stain_ord = stain_order.index(stain)
    file_meta = [slide, stain, stain_ord, coord, path]
    metadata.append(file_meta) 

time: 57.1 ms (started: 2022-01-06 17:02:04 -06:00)


In [51]:
# Filter all cropped images for first stain as baseline to register all other images to
stain0_list = [item for item in metadata if item[1] == stain_order[0]]

time: 6.18 ms (started: 2022-01-06 17:02:04 -06:00)


In [52]:
# For each cropped stain_order[0] image, find all files that are from the matching row - col location of the slide.  
# If a matching crop does not exist in all files, eliminate.
# Creates dict with key = stain_order[0] path and values = all matching tile paths in stain order
# Creates list of all stain_order[0] paths where all stain channel images exist to iterate through
all_paths = {}
l_stain0_filter = []
for coord in stain0_list:
    files = [item for item in metadata if (item[0] == coord[0] and item[3] == coord[3])]
    if len(files) == len(stain_order):
        for i in files:
            files = sorted(files, key = lambda x: x[2])
            if i[1] == stain_order[0]:
                l_stain0_filter.append(i[4])
            file_paths = [item[4] for item in files]
            key = [item[4] for item in files if item[1] == stain_order[0]]
            all_paths[key[0]] = file_paths
l_stain0_filter = sorted(l_stain0_filter)

time: 146 ms (started: 2022-01-06 17:02:05 -06:00)


In [53]:
regAll(l_stain0_filter[0])

time: 2min 5s (started: 2022-01-06 17:02:05 -06:00)


In [54]:
# registers all image sets per tile
Parallel(n_jobs = int(available_cores * 0.9))(delayed(regAll)(file)for file in tqdm(l_stain0_filter))


100%|██████████| 387/387 [4:23:49<00:00, 40.90s/it]  


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

time: 5h 21min 46s (started: 2022-01-06 17:04:10 -06:00)


In [ ]:
# dev mode open random selection of registered stacks to visually confirm successful registration


### Filter for only "Good" Tiles, Crop Stacks + Stitch

In [ ]:
good_tiles = pd.read_csv(output_dir + '/good_tiles.csv', header = None)   # save list of good tiles as good_tiles.csv file in the base output directory.  Column 1 = Row, Column 2 = Column, Column 3 = Slide
tiles_in = []
for x in good_tiles.values.tolist():
    tiles_in.append((x[0], x[1], x[2]))

In [ ]:
# create list of all 'stack' files
l_stacks = []
for root, dirs, files in os.walk(reg_folder):
    for file in files:
        if file.endswith('_stack.tiff'):
            l_stacks.append(os.path.join(root, file))

In [ ]:
sort_l_files = [file for file in l_files if os.path.basename(file).startswith('Aligned_0')]   #list of 1 image from each patient

In [ ]:
for p in sort_l_files:
    _, _, _, slide_ID, _ = os.path.basename(p).split('_')

    stack_meta = {}
    for path in l_stacks:
        file = os.path.basename(os.path.splitext(path)[0])
        slide,row, col, _ = file.split('_')
        int_row = row[1:]
        int_col = col[1:]
        coord = (int(int_row), int(int_col), str(slide))
        if slide == slide_ID:
            if coord in tiles_in:                 # Checks if each coord is listed in users's good_tiles list.  Can comment out this line if all tiles are considered 'good'
                stack_meta[coord] = path

    img = io.imread(p)   # reads image shape without reading full image
    num_rows = list(range(int(img.shape[0]/tile_x)))
    num_cols = list(range(int(img.shape[1]/tile_y)))
    combined = [(f,s, slide) for f in num_rows for s in num_cols] 

    l_CropPath = []
    for item in combined:
        if item in stack_meta.keys():
            l_CropPath.append((item, stack_meta[item]))
        else:
            l_CropPath.append((item, 'placeholder'))

    l_crops = []
    for item in l_CropPath:
        if item[1] == 'placeholder':
            l_crops.append(np.zeros((len(stain_order),tile_x,tile_y,3), dtype = np.uint8))
        else:
            row = item[0][0]
            col = item[0][1]
            if row == 0:
                row_top_offset = 0
            else:
                row_top_offset = overlap_x
            if row == int(img.shape[0]/tile_x)-1:
                row_bot_offset = 0
            else:
                row_bot_offset = overlap_x
            if col == 0:
                col_l_offset = 0
            else:
                col_l_offset = overlap_y
            if col == int(img.shape[1]/tile_y)-1:
                col_r_offset = 0
            else:
                col_r_offset = overlap_y
            cropped_img = crop(io.imread(item[1]), ((0,0), (row_top_offset, row_bot_offset), (col_l_offset, col_r_offset), (0,0)), copy = False)
            l_crops.append(cropped_img)
            assert cropped_img.shape == (len(stain_order),tile_x, tile_y,3)
            tifffile.imwrite(crops_folder + '/{}_r{}_c{}_cropped.tiff'.format(item[0][2],item[0][0], item[0][1]), cropped_img)    # crop off overlap pixels and save as final registered stack tiles
    conc_rows = []
    for x in range(len(num_rows)):
        conc_rows.append((np.concatenate(l_crops[((max(num_cols)+1)*(x)):((max(num_cols)+1)*(x+1))], axis = 2)))
    conc_all = np.concatenate(conc_rows, axis = 1)

    tifffile.imwrite(stitch_folder + f'/{slide_ID}_stitched.tiff', conc_all)   # stitch all stacks together for visualization

### Napari Visualization of Stitched Stacks

In [ ]:
# OPTIONAL and depending on PC specs may not work with insufficent memory

In [ ]:
l_full_stitch = []

for root, dirs, files in os.walk(output_dir + '/StitchedSlides/'):
    for file in files:
        if file.endswith('_stitched.tiff'):
            l_full_stitch.append(os.path.join(root, file))

In [ ]:
import napari
if dev_mode == True:
    file = l_full_stitch[0]   # edit int value to choose which slide to visualize

    with napari.gui_qt():
        viewer = napari.view_image(tifffile.imread(file), name = os.path.basename(file))

### Stardist Segmentation of Tiles and Readout Measurements

In [ ]:
def deconv(image):
    if type(image) == str:
        img = io.imread(image)
        hax  = separate_stains(img, hax_from_rgb)
        hema = hax[:,:,0]
        h = rescale_intensity(hema, in_range = (0, 1), out_range = 'uint8')   # edit in-range for better isolation of hematoxylin signal
        
        aec = hax[:,:,1]
        a = rescale_intensity(aec, in_range = (0,1), out_range = 'uint8')
    elif type(image) == np.ndarray:
        hax = separate_stains(image, hax_from_rgb)
        hema = hax[:,:,0]
        h = rescale_intensity(hema, in_range = (0, 1), out_range = 'uint8')
        
        aec = hax[:,:,1]
        a = rescale_intensity(aec, in_range = (0,1), out_range = 'uint8')
    return h, a


In [ ]:
os.chdir(stardist_dir) 
model = StarDist2D(None, name='stardist_7_13', basedir='models')

axis_norm = (0,1)
n_channel = 1


In [ ]:
l_cropped_stacks = []   # list of dicts, single dict per slide, within dict:  tile_x, tile_y, slide_id  =  stack array
for x in slide_id:
    d_cropped_stacks = {}
    for root, dirs, files in os.walk(crops_folder):
        for i in files:
            if i.endswith('_cropped.tiff'):
                if i.startswith(x):
                    array = io.imread(os.path.join(root,i))
                    pat, row, col, _ = i.split('_')
                    tile = (int(row[1:]), int(col[1:]), pat)
                    d_cropped_stacks[tile] = array
    l_cropped_stacks.append(d_cropped_stacks)

In [ ]:
# code for reading labels from external source
if batch_mode == False: 
    if stardist_segmentation == True:
        for y in [l_cropped_stacks[test_slide_index]]:    #  to only compute for 1 slide
            df_all = pd.DataFrame()
            d_labels = {}
            d_expanded_labels = {}

            for tile, array in y.items():
                df_int = pd.DataFrame()
                seg = deconv(array[4])
                norm = normalize(seg[0], 1, 99.8, axis = axis_norm)
                labels, details = model.predict_instances(norm)

                d_labels[tile] = labels

                exp_lab = expand_labels(labels, distance = exp_lab_dist)   

                d_expanded_labels[tile] = exp_lab

                #save labels
                io.imsave(labels_folder + '/{}_r{}_c{}_labels.tiff'.format(tile[2], tile[0], tile[1]), labels)
                if dev_mode == True:
                    io.imsave(labels_folder + '/{}_r{}_c{}_seg.tiff'.format(tile[2], tile[0], tile[1]), seg[0])   # saves deconvoluted image that stardist works on
                    io.imsave(labels_folder + '/{}_r{}_c{}_seg_raw.tiff'.format(tile[2], tile[0], tile[1]), array[5])  # saves raw pre-deconvolution image 


                for i in range(array.shape[0]):
                    int_img = deconv(array[i])[1]
                    nuc_props = regionprops_table(label_image = labels, intensity_image = int_img, 
                                              properties = ('label', 'centroid', 'mean_intensity'))
                    nuc_data = pd.DataFrame(nuc_props)
                    nuc_data.rename(columns = {'mean_intensity': 'mean_intensity_' + stain_order[i] + '_nuc'}, inplace = True)

                    cell_props = regionprops_table(label_image = exp_lab, intensity_image = int_img, 
                                              properties = ('label', 'mean_intensity'))
                    cell_data = pd.DataFrame(cell_props)
                    cell_data.rename(columns = {'mean_intensity': 'mean_intensity_' + stain_order[i] + '_cell'}, inplace = True)

                    data = nuc_data.merge(cell_data, on = 'label')

                    if i != 0:
                        data = data.drop(['centroid-1', 'centroid-0'], axis = 1)

                    if i == 0:
                        data.insert(1, 'Tile', str(tile))
                        data.insert(1, 'Tile_row', tile[0])
                        data.insert(2, 'Tile_col', tile[1])
                        data['Global_y'] = data['centroid-0']+(data['Tile_row']*float(tile_x))
                        data['Global_x'] = data['centroid-1']+(data['Tile_col']*float(tile_y))
                        df_int = data
                    else:
                        df_int = df_int.merge(data, on = 'label')
                try:
                    df_all = df_all.append(df_int, ignore_index = True)
                except:
                    df_all = df_int
            df_all.to_csv(output_dir + '/{}_export.csv'.format(tile[2]))
    else:
        d_labels = {}
        for ii in glob.glob(labels_folder + '/*labels.tiff'):
            pat, row, col, _ = os.path.basename(ii).split('_')
            tile = (int(row[1:]), int(col[1:]), pat)
        for y in [l_cropped_stacks[test_slide_index]]:
            df_all = pd.DataFrame()

            for tile, array in y.items():
                labels = d_labels[tile]
                exp_lab = expand_labels(labels, distance = exp_lab_dist)   

                for i in range(array.shape[0]):
                    int_img = deconv(array[i])[1]
                    nuc_props = regionprops_table(label_image = labels, intensity_image = int_img, 
                                              properties = ('label', 'centroid', 'mean_intensity'))
                    nuc_data = pd.DataFrame(nuc_props)
                    nuc_data.rename(columns = {'mean_intensity': 'mean_intensity_' + stain_order[i] + '_nuc'}, inplace = True)

                    cell_props = regionprops_table(label_image = exp_lab, intensity_image = int_img, 
                                              properties = ('label', 'mean_intensity'))
                    cell_data = pd.DataFrame(cell_props)
                    cell_data.rename(columns = {'mean_intensity': 'mean_intensity_' + stain_order[i] + '_cell'}, inplace = True)

                    data = nuc_data.merge(cell_data, on = 'label')

                    if i != 0:
                        data = data.drop(['centroid-1', 'centroid-0'], axis = 1)

                    if i == 0:
                        data.insert(1, 'Tile', str(tile))
                        data.insert(1, 'Tile_row', tile[0])
                        data.insert(2, 'Tile_col', tile[1])
                        data['Global_y'] = data['centroid-0']+(data['Tile_row']*float(tile_x))
                        data['Global_x'] = data['centroid-1']+(data['Tile_col']*float(tile_y))
                        df_int = data
                    else:
                        df_int = df_int.merge(data, on = 'label')
                try:
                    df_all = df_all.append(df_int, ignore_index = True)
                except:
                    df_all = df_int
            df_all.to_csv(output_dir + '/{}_export.csv'.format(tile[2]))
else:
    if stardist_segmentation == True:
        for y in l_cropped_stacks:  
            df_all = pd.DataFrame()
            d_labels = {}
            d_expanded_labels = {}

            for tile, array in y.items():
                df_int = pd.DataFrame()
                seg = deconv(array[4])
                norm = normalize(seg[0], 1, 99.8, axis = axis_norm)
                labels, details = model.predict_instances(norm)

                d_labels[tile] = labels

                exp_lab = expand_labels(labels, distance = exp_lab_dist)   

                d_expanded_labels[tile] = exp_lab

                #save labels
                io.imsave(labels_folder + '/{}_r{}_c{}_labels.tiff'.format(tile[2], tile[0], tile[1]), labels)
                if dev_mode == True:
                    io.imsave(labels_folder + '/{}_r{}_c{}_seg.tiff'.format(tile[2], tile[0], tile[1]), seg[0])   # saves deconvoluted image that stardist works on
                    io.imsave(labels_folder + '/{}_r{}_c{}_seg_raw.tiff'.format(tile[2], tile[0], tile[1]), array[5])  # saves raw pre-deconvolution image 


                for i in range(array.shape[0]):
                    int_img = deconv(array[i])[1]
                    nuc_props = regionprops_table(label_image = labels, intensity_image = int_img, 
                                              properties = ('label', 'centroid', 'mean_intensity'))
                    nuc_data = pd.DataFrame(nuc_props)
                    nuc_data.rename(columns = {'mean_intensity': 'mean_intensity_' + stain_order[i] + '_nuc'}, inplace = True)

                    cell_props = regionprops_table(label_image = exp_lab, intensity_image = int_img, 
                                              properties = ('label', 'mean_intensity'))
                    cell_data = pd.DataFrame(cell_props)
                    cell_data.rename(columns = {'mean_intensity': 'mean_intensity_' + stain_order[i] + '_cell'}, inplace = True)

                    data = nuc_data.merge(cell_data, on = 'label')

                    if i != 0:
                        data = data.drop(['centroid-1', 'centroid-0'], axis = 1)

                    if i == 0:
                        data.insert(1, 'Tile', str(tile))
                        data.insert(1, 'Tile_row', tile[0])
                        data.insert(2, 'Tile_col', tile[1])
                        data['Global_y'] = data['centroid-0']+(data['Tile_row']*float(tile_x))
                        data['Global_x'] = data['centroid-1']+(data['Tile_col']*float(tile_y))
                        df_int = data
                    else:
                        df_int = df_int.merge(data, on = 'label')
                try:
                    df_all = df_all.append(df_int, ignore_index = True)
                except:
                    df_all = df_int
            df_all.to_csv(output_dir + '/{}_export.csv'.format(tile[2]))
        else:
            d_labels = {}
            for ii in glob.glob(labels_folder + '/*labels.tiff'):
                pat, row, col, _ = os.path.basename(ii).split('_')
                tile = (int(row[1:]), int(col[1:]), pat)
            for y in l_cropped_stacks:
                df_all = pd.DataFrame()

                for tile, array in y.items():
                    labels = d_labels[tile]
                    exp_lab = expand_labels(labels, distance = exp_lab_dist)   

                    for i in range(array.shape[0]):
                        int_img = deconv(array[i])[1]
                        nuc_props = regionprops_table(label_image = labels, intensity_image = int_img, 
                                                  properties = ('label', 'centroid', 'mean_intensity'))
                        nuc_data = pd.DataFrame(nuc_props)
                        nuc_data.rename(columns = {'mean_intensity': 'mean_intensity_' + stain_order[i] + '_nuc'}, inplace = True)

                        cell_props = regionprops_table(label_image = exp_lab, intensity_image = int_img, 
                                                  properties = ('label', 'mean_intensity'))
                        cell_data = pd.DataFrame(cell_props)
                        cell_data.rename(columns = {'mean_intensity': 'mean_intensity_' + stain_order[i] + '_cell'}, inplace = True)

                        data = nuc_data.merge(cell_data, on = 'label')

                        if i != 0:
                            data = data.drop(['centroid-1', 'centroid-0'], axis = 1)

                        if i == 0:
                            data.insert(1, 'Tile', str(tile))
                            data.insert(1, 'Tile_row', tile[0])
                            data.insert(2, 'Tile_col', tile[1])
                            data['Global_y'] = data['centroid-0']+(data['Tile_row']*float(tile_x))
                            data['Global_x'] = data['centroid-1']+(data['Tile_col']*float(tile_y))
                            df_int = data
                        else:
                            df_int = df_int.merge(data, on = 'label')
                    try:
                        df_all = df_all.append(df_int, ignore_index = True)
                    except:
                        df_all = df_int
                df_all.to_csv(output_dir + '/{}_export.csv'.format(tile[2]))

In [ ]:
df_all.to_csv(output_dir + '/df_all.csv')    # save all features

In [ ]:
df_all = pd.read_csv(output_dir + '/df_all.csv')   # load all features.  Can skip to this step if above has already been run and only visualizations below are needed

In [ ]:
df_all

### Whole Slide Psudocolor Plots

In [ ]:
for column in df_all.columns:
    print(column)
feature = 'mean_intensity_CD68_cell'    # string for specific feature of interest. Copy and paste from printed strings below

In [ ]:
# figure of all detections scatter of full slide
if batch_mode == False:
    plt.figure(figsize=(20,20))
    ax = sns.scatterplot(data = df_all, x = 'Global_x', y = 'Global_y', hue = feature, hue_norm = (0, 100), palette = 'viridis', linewidth=0, s =10)
    ax.set_aspect(aspect = 1)
    plt.gca().invert_yaxis()
    plt.savefig(f'{plots_folder}/{feature}_{slide_id[test_slide_index]}.pdf', dpi=500)

In [ ]:
if batch_mode == False:
    df_heatmap = df_all.groupby(['Tile']).mean(feature)[['Tile_row', 'Tile_col', feature]]
    df_heatmap['Tile_row'] = df_heatmap['Tile_row'].astype(int)
    df_heatmap['Tile_col'] = df_heatmap['Tile_col'].astype(int)
    
    plt.figure(figsize=(8,8))

    ax = sns.scatterplot(data = df_heatmap, y = 'Tile_row', x = 'Tile_col', hue = feature, palette = 'viridis', 
                         s = 600, marker = 's')
    plt.gca().invert_yaxis()
    plt.legend(bbox_to_anchor = (0,1), loc = 'upper left', ncol =1)
    plt.title(f'Per Tile Expression {feature}')
    ax.set_aspect(aspect = 1)
    plt.savefig(f'{plots_folder}/Per_Tile_{feature}_{slide_id[test_slide_index]}.pdf')

### Voronoi Demo

In [ ]:
# Create voronoi diagram for selected tile with each cell polygon's color indicating the measured intensity of a feature of interest
# Next cell will list out available options for tile selection and feature selection 

In [ ]:
for tile in sorted(df_all['Tile'].unique()):
    print(tile)

for column in df_all.columns:
    print(column)

In [ ]:
tile_in = "(1, 3, 'N16-244-2A')"       # string for specific tile of interest. Make sure string perfectly matches
feature = 'mean_intensity_PD-L1_nuc'    # string for specific feature of interest. Make sure string perfectly matches

In [ ]:
df_tile = df_all[df_all['Tile'] == tile_in].reset_index()

coords = np.asarray(df_tile[['centroid-1', 'centroid-0']])


In [ ]:
from scipy.spatial import Delaunay, Voronoi, voronoi_plot_2d

vor = Voronoi(coords)

In [ ]:
import matplotlib as mpl
import matplotlib.cm as cm
fig = voronoi_plot_2d(vor, show_vertices = False, show_points = False)
fig.set_size_inches(10,10)
c = df_tile[feature]
minima = min(c)
maxima = max(c)

norm = mpl.colors.Normalize(vmin=minima, vmax=maxima, clip=True)
mapper = cm.ScalarMappable(norm=norm, cmap=cm.viridis)


for r in range(len(vor.point_region)):
    region = vor.regions[vor.point_region[r]]
    if not -1 in region:
        polygon = [vor.vertices[i] for i in region]
        plt.fill(*zip(*polygon), color=mapper.to_rgba(c[r]))
        
plt.ylim(0,2048)
plt.xlim(0, 2048)

plt.savefig(f'{plots_folder}/Vor_demo{feature}.pdf')